In [1]:
import re
from re import search
from nltk.corpus import stopwords
import string
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
# from pysentimiento import create_analyzer
# from pysentimiento.preprocessing import preprocess_tweet
# sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
# emotion_analyzer = create_analyzer(task="emotion", lang="es")
# hate_analyzer = create_analyzer(task="hate_speech", lang="es")
# from unidecode import unidecode

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("pysentimiento/robertuito-sentiment-analysis")

model = AutoModelForSequenceClassification.from_pretrained("pysentimiento/robertuito-sentiment-analysis")

ModuleNotFoundError: No module named 'transformers'

In [ ]:
df = pd.read_excel('data/export1.xlsx')

In [ ]:
df['date'] = pd.to_datetime(df['fecha'])
df['hour'] = df['date'].dt.hour
df['date2'] = df['date'].dt.date
df = df[['texto', 'date2', 'hour']]
df.columns = ['tweet', 'date', 'hour']

In [ ]:
name = 'Mall_Plaza'
df_tweets = df
df_tweets

In [ ]:
def gen_df(df, text, location, date, user):
    df['tweet'] = df[text].astype(str)
    column_list = ['tweet']
    if location != "":
        df['location'] = df[location].astype(str)
        column_list.append('location')
    if date != "":
        df['date'] = df[date]
        column_list.append('date')
    if user != "":
        df['user'] = df[user].astype(str)
        column_list.append('user')
    df_tweets = df[column_list]
    return df_tweets

# name = 'Lavin'
# df_tweets_muni = pd.read_csv("data//rs_dat.csv") #Lavin #text #no location #no date
# df_tweets = gen_df(df_tweets_muni, 'text', '', '', '') #Lavin

name = 'Generadoras2'
df_tweets_muni = pd.read_csv("data//TweetsGeneradoras.csv") #Generadoras2
df_tweets = gen_df(df_tweets_muni, 'texto_1', 'localizacion1', 'fecha', 'usuario') #Generadoras2

# name = 'Generadoras'
# df_tweets_muni = pd.read_csv("data//tweets_generadoras.csv") #Generadoras
# df_tweets = gen_df(df_tweets_muni, 'texto_1', 'localizacion1', 'fecha', 'usuario') #Generadoras

# name = 'Alto_Maipo'
# df_tweets_muni = pd.read_csv("data//rs_tweets.csv") #Alto Maipo
# df_tweets = gen_df(df_tweets_muni, 'strtext', 'user_location', 'created_at', 'user_name') #Alto Maipo

# name = 'Codelco'
# df_tweets_muni = pd.read_csv("data//rs_codelco_tweets.csv") #Codelco
# df_tweets = gen_df(df_tweets_muni, 'strtext', 'user_location', 'created_at', 'user_name') #Codelco

# len(df_tweets)

In [ ]:
# Cargar stopwords en español
stopwords_es = stopwords.words('spanish')

"""
Excluir menciones, emails, URLs y simbolos
"""
def clean_tweet(tweet):
    # Convertir a minusculas
    tweet = tweet.lower()

    # Excluir menciones o emails
    tweet = re.sub(r'\w*@(\w+\.*\w+\.*\w+)',' ', tweet)

    # Excluir simbolos
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Excluir URLs
    tweet = re.sub(r'(?:www\.|https?)[^\s]+', ' ', tweet, flags=re.MULTILINE)

    # Borrar espacios
    tweet = tweet.strip()

    # Considerar solo valores alfa numericos
    tweet_alfa = re.compile("^(?![0-9]*$)[a-zA-Z0-9]+$")

    # Eliminar stopwords y palabras con longitud <= 2
    tokens = tweet.split()
    text = [token for token in tokens if token not in stopwords_es and len(token)>2 and tweet_alfa.match(token)]
    return ' '.join(text)

# Aplicar filtro a tweets
df_tweets['tweet_cleaned'] = df_tweets['tweet'].apply(lambda x: clean_tweet(str(x)))
df_tweets['tweet_cleaned'] = df_tweets['tweet_cleaned'].apply(lambda x: preprocess_tweet(x))

In [ ]:
#Identificación de subtemas dentro de los temas
df_topic = pd.read_excel("risk.xlsx", sheet_name="versión 3")
df_filtro = df_topic.loc[0,:].reset_index()
df_filtro.columns = ['tema', 'subtema']
tema = df_filtro.tema.to_list()
for i, t in enumerate(tema):
    if 'Unnamed:' in t:
        tema[i] = tema[i-1]
df_filtro.tema = tema
df_filtro = df_filtro.dropna()

In [ ]:
#Identificación de palabras clave por subtema
df_topic = pd.read_excel("risk.xlsx", sheet_name="versión 3", header=[1])
for i in range(len(df_topic.columns)):
    subtema = (df_topic.columns[i])
    words = df_topic[df_topic.columns[i]].dropna().to_list()
    words = [w.lower() for w in words]
    pattern = '|'.join(words)
    topic = df_filtro[df_filtro['subtema'] == str(subtema)].tema.to_list()[0]
    column_name = 'isin_' + str(df_topic.columns[i]) + '_' + str(topic)
    df_tweets[column_name] = df_tweets['tweet_cleaned'].str.contains(pattern)

In [ ]:
#Identificar los temas mencionados en los tweets de acuerdo a palabras clave
columns = df_tweets.columns
columns_subtopic = [lis for lis in columns if 'isin' in lis]
is_relevant = []
for i in range(len(df_tweets)):
    is_relevant.append(any(df_tweets[columns_subtopic].loc[i,:].to_list()))
df_tweets['relevant'] = is_relevant

In [ ]:
#Identificación de sentimiento
df_tweets['sentiment'] = df_tweets['tweet_cleaned'].apply(lambda x: sentiment_analyzer.predict(x).output)
# df_tweets['hate'] = df_tweets['tweet_cleaned'].apply(lambda x: hate_analyzer.predict(x).output)

In [ ]:
#Identificación de emoción
df_tweets['emotion'] = df_tweets['tweet_cleaned'].apply(lambda x: emotion_analyzer.predict(x).output)

In [ ]:
#Generar un dataframe por subtema
sentiment_options = ['POS', 'NEG', 'NEU']
sentiment_list = []
for subtopic in columns_subtopic:
    aux = df_tweets[df_tweets[subtopic]==True].sentiment.value_counts().reindex(sentiment_options).fillna(0).to_list()
    sentiment_list.append(aux)
df_subtema = pd.DataFrame(sentiment_list, index = columns_subtopic, columns = sentiment_options).reset_index()
df_subtema[['aux', 'subtema', 'tema']] = df_subtema['index'].str.split('_', expand=True)

In [ ]:
#Generar un dataframe de tema y el indice de positividad o negatividad
df_tema = df_subtema.groupby(['tema']).sum()
df_tema['Total']= df_tema[sentiment_options].sum(axis=1)
df_tema['pos_index'] = df_tema.POS / df_tema.Total
df_tema['neg_index'] = df_tema.NEG / df_tema.Total
df_tema = df_tema.transpose().reset_index()

In [ ]:
def remove_accents(a):
    return unidecode.unidecode(a.decode('utf-8'))

In [ ]:
#Normalizar ubicación
df_tweets['location_clean'] = df_tweets['location'].apply(lambda x: preprocess_tweet(x))
df_tweets['location_clean'] = df_tweets['location_clean'].str.lower()
df_tweets['location_clean'] = df_tweets['location_clean'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
#Obtener Región si se menciona a una comuna asociada
df_location = pd.read_csv('locations.csv', sep = ";")
df_location['comuna'] = df_location['comuna'].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')

df_aux = df_location.groupby('región')['comuna'].apply(lambda x: "[%s]" % ', '.join(x))
def get_location(x):
    j = 0
    for i in range(len(df_aux)):
        region = df_aux.index[i]
        lista_comunas = df_aux[i].strip('][').split(', ')
        for i,comuna in enumerate(lista_comunas):
            if search(comuna, x):
                j =+ 1
                location = region
                return location
df_tweets['region'] = df_tweets['location_clean'].apply(lambda x: get_location(x))

In [ ]:
#Guardar los resultados en un csv
folder = 'Results//'
df_tweets.to_excel(folder + name + '_tweets.xlsx' , encoding='utf8', index=False)
df_subtema.to_excel(folder + name + '_subtema.xlsx' , encoding='utf8', index=False)
df_tema.to_excel(folder + name + '_tema.xlsx' , encoding='utf8', index=False)

In [ ]:
df_tweets.groupby(['date', 'hour']).sum()

In [ ]:
item = 'isin_Covid_Salud'
sentiment_options = ['POS', 'NEG', 'NEU']
df_aux = df_tweets[df_tweets[item] == True][['date', 'hour', 'sentiment']]
df_aux['date'] = pd.to_datetime(df_aux['date'], format='%Y-%m-%d')
df_aux['hour'] = pd.to_datetime(df_aux['hour'], format='%H')
df_aux['time'] = pd.to_datetime(df_aux.date.astype(str) + ' ' + df_aux.hour.astype(str)).dt.strftime("%Y-%m-%d %H:%M:%S")
pd.get_dummies(df_aux, columns = ['sentiment']).groupby('time').sum()

In [ ]:
#Generar un dataframe por subtema
sentiment_options = ['POS', 'NEG', 'NEU']
sentiment_list = []
for subtopic in columns_subtopic:
    aux = df_tweets[df_tweets[subtopic]==True].sentiment.value_counts().reindex(sentiment_options).fillna(0).to_list()
    sentiment_list.append(aux)
df_subtema = pd.DataFrame(sentiment_list, index = columns_subtopic, columns = sentiment_options).reset_index()
df_subtema[['aux', 'subtema', 'tema']] = df_subtema['index'].str.split('_', expand=True)

In [ ]:
df_subtema

In [ ]:
df['date'] = pd.to_datetime(df['fecha'])
df['hour'] = df['date'].dt.hour
df['date2'] = df['date'].dt.date
df = df[['texto', 'date2', 'hour']]
df.columns = ['tweet', 'date', 'hour']

In [ ]:
name = 'Mall_Plaza'
df_tweets = df
df_tweets

In [ ]:
def gen_df(df, text, location, date, user):
    df['tweet'] = df[text].astype(str)
    column_list = ['tweet']
    if location != "":
        df['location'] = df[location].astype(str)
        column_list.append('location')
    if date != "":
        df['date'] = df[date]
        column_list.append('date')
    if user != "":
        df['user'] = df[user].astype(str)
        column_list.append('user')
    df_tweets = df[column_list]
    return df_tweets

# name = 'Lavin'
# df_tweets_muni = pd.read_csv("data//rs_dat.csv") #Lavin #text #no location #no date
# df_tweets = gen_df(df_tweets_muni, 'text', '', '', '') #Lavin

name = 'Generadoras2'
df_tweets_muni = pd.read_csv("data//TweetsGeneradoras.csv") #Generadoras2
df_tweets = gen_df(df_tweets_muni, 'texto_1', 'localizacion1', 'fecha', 'usuario') #Generadoras2

# name = 'Generadoras'
# df_tweets_muni = pd.read_csv("data//tweets_generadoras.csv") #Generadoras
# df_tweets = gen_df(df_tweets_muni, 'texto_1', 'localizacion1', 'fecha', 'usuario') #Generadoras

# name = 'Alto_Maipo'
# df_tweets_muni = pd.read_csv("data//rs_tweets.csv") #Alto Maipo
# df_tweets = gen_df(df_tweets_muni, 'strtext', 'user_location', 'created_at', 'user_name') #Alto Maipo

# name = 'Codelco'
# df_tweets_muni = pd.read_csv("data//rs_codelco_tweets.csv") #Codelco
# df_tweets = gen_df(df_tweets_muni, 'strtext', 'user_location', 'created_at', 'user_name') #Codelco

# len(df_tweets)

In [ ]:
# Cargar stopwords en español
stopwords_es = stopwords.words('spanish')

"""
Excluir menciones, emails, URLs y simbolos
"""
def clean_tweet(tweet):
    # Convertir a minusculas
    tweet = tweet.lower()

    # Excluir menciones o emails
    tweet = re.sub(r'\w*@(\w+\.*\w+\.*\w+)',' ', tweet)

    # Excluir simbolos
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Excluir URLs
    tweet = re.sub(r'(?:www\.|https?)[^\s]+', ' ', tweet, flags=re.MULTILINE)

    # Borrar espacios
    tweet = tweet.strip()

    # Considerar solo valores alfa numericos
    tweet_alfa = re.compile("^(?![0-9]*$)[a-zA-Z0-9]+$")

    # Eliminar stopwords y palabras con longitud <= 2
    tokens = tweet.split()
    text = [token for token in tokens if token not in stopwords_es and len(token)>2 and tweet_alfa.match(token)]
    return ' '.join(text)

# Aplicar filtro a tweets
df_tweets['tweet_cleaned'] = df_tweets['tweet'].apply(lambda x: clean_tweet(str(x)))
df_tweets['tweet_cleaned'] = df_tweets['tweet_cleaned'].apply(lambda x: preprocess_tweet(x))

In [ ]:
#Identificación de subtemas dentro de los temas
df_topic = pd.read_excel("risk.xlsx", sheet_name="versión 3")
df_filtro = df_topic.loc[0,:].reset_index()
df_filtro.columns = ['tema', 'subtema']
tema = df_filtro.tema.to_list()
for i, t in enumerate(tema):
    if 'Unnamed:' in t:
        tema[i] = tema[i-1]
df_filtro.tema = tema
df_filtro = df_filtro.dropna()

In [ ]:
#Identificación de palabras clave por subtema
df_topic = pd.read_excel("risk.xlsx", sheet_name="versión 3", header=[1])
for i in range(len(df_topic.columns)):
    subtema = (df_topic.columns[i])
    words = df_topic[df_topic.columns[i]].dropna().to_list()
    words = [w.lower() for w in words]
    pattern = '|'.join(words)
    topic = df_filtro[df_filtro['subtema'] == str(subtema)].tema.to_list()[0]
    column_name = 'isin_' + str(df_topic.columns[i]) + '_' + str(topic)
    df_tweets[column_name] = df_tweets['tweet_cleaned'].str.contains(pattern)

In [ ]:
#Identificar los temas mencionados en los tweets de acuerdo a palabras clave
columns = df_tweets.columns
columns_subtopic = [lis for lis in columns if 'isin' in lis]
is_relevant = []
for i in range(len(df_tweets)):
    is_relevant.append(any(df_tweets[columns_subtopic].loc[i,:].to_list()))
df_tweets['relevant'] = is_relevant

In [ ]:
#Identificación de sentimiento
df_tweets['sentiment'] = df_tweets['tweet_cleaned'].apply(lambda x: sentiment_analyzer.predict(x).output)
# df_tweets['hate'] = df_tweets['tweet_cleaned'].apply(lambda x: hate_analyzer.predict(x).output)

In [ ]:
#Identificación de emoción
df_tweets['emotion'] = df_tweets['tweet_cleaned'].apply(lambda x: emotion_analyzer.predict(x).output)

In [ ]:
#Generar un dataframe por subtema
sentiment_options = ['POS', 'NEG', 'NEU']
sentiment_list = []
for subtopic in columns_subtopic:
    aux = df_tweets[df_tweets[subtopic]==True].sentiment.value_counts().reindex(sentiment_options).fillna(0).to_list()
    sentiment_list.append(aux)
df_subtema = pd.DataFrame(sentiment_list, index = columns_subtopic, columns = sentiment_options).reset_index()
df_subtema[['aux', 'subtema', 'tema']] = df_subtema['index'].str.split('_', expand=True)

In [ ]:
#Generar un dataframe de tema y el indice de positividad o negatividad
df_tema = df_subtema.groupby(['tema']).sum()
df_tema['Total']= df_tema[sentiment_options].sum(axis=1)
df_tema['pos_index'] = df_tema.POS / df_tema.Total
df_tema['neg_index'] = df_tema.NEG / df_tema.Total
df_tema = df_tema.transpose().reset_index()

In [ ]:
def remove_accents(a):
    return unidecode.unidecode(a.decode('utf-8'))

In [ ]:
#Normalizar ubicación
df_tweets['location_clean'] = df_tweets['location'].apply(lambda x: preprocess_tweet(x))
df_tweets['location_clean'] = df_tweets['location_clean'].str.lower()
df_tweets['location_clean'] = df_tweets['location_clean'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [ ]:
#Obtener Región si se menciona a una comuna asociada
df_location = pd.read_csv('locations.csv', sep = ";")
df_location['comuna'] = df_location['comuna'].str.normalize('NFKD').str.encode('ascii',errors='ignore').str.decode('utf-8')

df_aux = df_location.groupby('región')['comuna'].apply(lambda x: "[%s]" % ', '.join(x))
def get_location(x):
    j = 0
    for i in range(len(df_aux)):
        region = df_aux.index[i]
        lista_comunas = df_aux[i].strip('][').split(', ')
        for i,comuna in enumerate(lista_comunas):
            if search(comuna, x):
                j =+ 1
                location = region
                return location
df_tweets['region'] = df_tweets['location_clean'].apply(lambda x: get_location(x))

In [ ]:
#Guardar los resultados en un csv
folder = 'Results//'
df_tweets.to_excel(folder + name + '_tweets.xlsx' , encoding='utf8', index=False)
df_subtema.to_excel(folder + name + '_subtema.xlsx' , encoding='utf8', index=False)
df_tema.to_excel(folder + name + '_tema.xlsx' , encoding='utf8', index=False)

In [ ]:
df_tweets.groupby(['date', 'hour']).sum()

In [ ]:
item = 'isin_Covid_Salud'
sentiment_options = ['POS', 'NEG', 'NEU']
df_aux = df_tweets[df_tweets[item] == True][['date', 'hour', 'sentiment']]
df_aux['date'] = pd.to_datetime(df_aux['date'], format='%Y-%m-%d')
df_aux['hour'] = pd.to_datetime(df_aux['hour'], format='%H')
df_aux['time'] = pd.to_datetime(df_aux.date.astype(str) + ' ' + df_aux.hour.astype(str)).dt.strftime("%Y-%m-%d %H:%M:%S")
pd.get_dummies(df_aux, columns = ['sentiment']).groupby('time').sum()

In [ ]:
#Generar un dataframe por subtema
sentiment_options = ['POS', 'NEG', 'NEU']
sentiment_list = []
for subtopic in columns_subtopic:
    aux = df_tweets[df_tweets[subtopic]==True].sentiment.value_counts().reindex(sentiment_options).fillna(0).to_list()
    sentiment_list.append(aux)
df_subtema = pd.DataFrame(sentiment_list, index = columns_subtopic, columns = sentiment_options).reset_index()
df_subtema[['aux', 'subtema', 'tema']] = df_subtema['index'].str.split('_', expand=True)

In [ ]:
df_subtema